In [1]:
import numpy as np
import scipy
import pandas as pd
import math
import random
import sklearn
import unicodedata

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder

from scipy.sparse.linalg import svds

import matplotlib.pyplot as plt

In [2]:
from keras.models import Sequential, Model
from keras.layers import Dense, Embedding, Input, Conv2D, Concatenate, MaxPool2D, Reshape, Flatten, Dropout
from keras.layers import ReLU
from keras.layers import BatchNormalization, LSTM, Bidirectional

from keras.callbacks import CSVLogger, EarlyStopping, ReduceLROnPlateau
from keras.layers import Add

from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [3]:
from keras.optimizers import Adam, SGD
from keras.callbacks import LearningRateScheduler
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from sklearn.manifold import TSNE
from keras import backend as K

In [4]:
# !python3 -m pip install gensim

In [5]:
from gensim.models import Word2Vec

In [6]:
%matplotlib inline

In [7]:
# !python3 -m pip install -U nltk
# !python3 -m nltk.downloader punkt

In [8]:
import nltk

In [9]:
from nltk.corpus import stopwords

In [10]:
path = 'recipes_with_calories_and_protein.csv'

In [11]:
df = pd.read_csv(path, sep="|", index_col=0)

In [12]:
all_columns = list(df.columns)
len(all_columns)

680

In [13]:
df.dropna(axis=0).shape, df.shape

((15867, 680), (20052, 680))

In [14]:
for column in df.columns:
    if df[column].isnull().any():
        print(column)
#         df[column] = df[column].fillna(df[column].mean())

fat
sodium


In [15]:
df.head()

,title,rating,calories,protein,fat,sodium,#cakeweek,#wasteless,22-minute meals,3-ingredient recipes,...,yellow squash,yogurt,yonkers,yuca,zucchini,cookbooks,leftovers,snack,snack week,turkey
0,"Lentil, Apple, and Turkey Wrap",2.500,426.000000,30.000000,7.0,559.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,Boudin Blanc Terrine with Red Onion Confit,4.375,403.000000,18.000000,23.0,1439.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Potato and Fennel Soup Hodge,3.750,165.000000,6.000000,7.0,165.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Mahi-Mahi in Tomato Olive Sauce,5.000,252.445801,14.630608,NaN,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Spinach Noodle Casserole,3.125,547.000000,20.000000,32.0,452.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
def get_small_dict(df, keep_columns, index):
    my_dict = df.iloc[index][keep_columns].to_dict()
    my_dict['tags'] = [
        key for key, value in df.iloc[index].to_dict().items() if value != 0 and key not in keep_columns]
    my_dict['title'] = my_dict['title'].strip()
    my_dict['tags'] = [
        str(unicodedata.normalize('NFKD', t).encode('ascii','ignore'), encoding='ascii') for t in my_dict['tags']
    ]
    return my_dict

In [17]:
keep_columns = ['title', 'rating', 'calories', 'protein', 'fat','sodium']

In [18]:
def remove_outliers_with_tukey(df):
    print('There were {} samples before outlier removal.'.format(len(df)))

    # Remove calories outliers by the Tukey rule
    Q3 = np.percentile(df[regr_column], 75)
    Q1 = np.percentile(df[regr_column], 25)
    IQR = Q3 - Q1

    outliers = df[regr_column].loc[
        (df[regr_column] < (Q1 - 1.5 * IQR)) | 
        (df[regr_column] > (Q3 + 1.5 * IQR))
    ].index
    
    df = df.drop(outliers)
    
    print('There are now {} samples.'.format(len(cut_df)))

    return df

In [20]:
# pick regression column
regr_column = 'fat'
cut_df = df[df[regr_column] == df[regr_column]]

cut_df = remove_outliers_with_tukey(df=cut_df)

# transform df in list of dictionaries for processing
regr_data = []
for i in range(len(cut_df)):
    regr_data.append(get_small_dict(cut_df, keep_columns, i))

There were 15869 samples before outlier removal.
There are now 15869 samples.


In [21]:
def remove_punctuation_signs(regr_data):
    for d in regr_data:
        if 'tags' not in d:
            raise ValueError('Expected to have <tags> key in each of the regression input data dictionary!')
    corpus = ['|'.join(
        d['tags'] + 
        [w.lower() for w in nltk.word_tokenize(d['title']) if
             w not in '''!()-[]{};:'"\,<>./?@#$%^&*_~''']
    ) for d in regr_data]
    return corpus

In [22]:
# make sure the data doesn't have any punctuation signs
corpus = remove_punctuation_signs(regr_data)

In [23]:
# pick tokenization method
def tokenize(word_list):
    return word_list.split('|')

In [24]:
def get_y_labels(regr_column, regr_data):
    return [d[regr_column] for d in regr_data]

In [25]:
y_label = get_y_labels(regr_column, regr_data)

In [26]:
if len(corpus) != len(y_label):
    print('[ERROR] There is a mismatch between the size of the input data ' +
          ' ({} elements) and the size of labels ({} elements)!'.format(len(corpus), len(y_label)))

## Try a vanilla regression (CountVectorizer + LinearRegression from sklearn)

In [28]:
# fit vectorizer
vectorizer = CountVectorizer(tokenizer=tokenize)
X = vectorizer.fit_transform(corpus)

# train, test data
X_train, X_test, y_train, y_test = train_test_split(X.toarray(), y_label, test_size=0.2, random_state=42)

# sklearn LinearRegression model
regr_sklearn = LinearRegression()
regr_sklearn.fit(X_train, y_train)

# predict data
y_pred = regr_sklearn.predict(X_test)

# how far off is the result? metric = mean squared error
y_test = np.array(y_test)
y_pred = np.array(y_pred)
print("Mean squared error: ", mean_squared_error(y_true=y_test, y_pred=y_pred))

# show first k differences
first_k = 10
for yp, yt in zip(y_pred[:first_k], y_test[:first_k]):
    print(yp, yt)

Mean squared error:  1.0175001141546378e+25
529199939986.90283 56.0
30.16865718261775 44.0
4.367280839844312 6.0
14.0084398974615 12.0
20.501459001465406 45.0
9.588830827637281 0.0
14.3924731005865 17.0
14.3761156787115 14.0
24.83369624511775 10.0
3058434.317991387 10.0


## Try Word2Vec embeddings + Keras NN

### Word2Vec embedding

In [29]:
vocab = set()
for word_list in corpus:
    vocab |= set(tokenize(word_list))

print("Choose embedding size between {} and {}".format(np.log2(len(vocab)), np.power(len(vocab), 1/4) * 10))

w2v_corpus = []
for word_list in corpus:
    w2v_corpus.append(tokenize(word_list))

Choose embedding size between 12.52820989662271 and 87.66812995049386


In [30]:
EMBEDDING_SIZE = 50

In [31]:
w2v_model = Word2Vec(w2v_corpus, size=EMBEDDING_SIZE, window=5, min_count=1, workers=-1)

print("Embedding size is {}".format(w2v_model.wv.vector_size))

Embedding size is 50


In [32]:
sequence_lengths = [len(w) for w in w2v_corpus]

print("Sequence length should be {} ".format(pd.Series(sequence_lengths).describe()['mean']))

Sequence length should be 17.027901936882028 


In [33]:
FIXED_LENGTH = 17

In [34]:
def generate_embeddings_matrix(sample, model, fixed_length=17):
    embedding_size = model.wv.vector_size
    embeddings = []
    for sen in sample:
        for word in sen:
            if word not in model.wv.vocab:
                continue
            embeddings.append(model.wv[word])
            if len(embeddings) >= fixed_length:
                break

    embeddings = np.array(embeddings)

    while len(embeddings) < fixed_length:
        padding_length = fixed_length - len(embeddings)
        embeddings = np.concatenate((embeddings, embeddings[:padding_length]))
    if len(embeddings) > fixed_length:
        embeddings = embeddings[:fixed_length]

    return embeddings

In [35]:
def generate_input_matrix(sample, fixed_length=17):
    
    inputs = np.array(sample)

    while len(inputs) < fixed_length:
        padding_length = fixed_length - len(inputs)
        inputs = np.concatenate((inputs, inputs[:padding_length]))
    if len(inputs) > fixed_length:
        inputs = inputs[:fixed_length]

    return inputs

In [36]:
# Generate list of input data (embedding matrix) and make sure every input has FIXED_LENGTH elements
X = []
for sample in w2v_corpus:
    embedded_sample = generate_embeddings_matrix(sample, w2v_model, FIXED_LENGTH)
    if embedded_sample.shape != (FIXED_LENGTH, EMBEDDING_SIZE):
        print(embedded_sample.shape)
    X.append(embedded_sample)

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y_label, test_size=0.2, random_state=42)

### Keras NN

In [38]:
FIXED_LENGTH = 17
EMBEDDING_SIZE = 50

BATCH_SIZE = 16
EPOCHS=200

In [39]:
def baseline_model():
    # Create model
    input_layer = Input(shape=(FIXED_LENGTH, EMBEDDING_SIZE), name='input')
    flat = Flatten()(input_layer)
    z = Dense(32, activation='relu', name='fc1', kernel_initializer='normal')(flat)
    output = Dense(1, kernel_initializer='normal')(z)
    model = Model(inputs=[input_layer], outputs=[output])
    # Compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [40]:
model = baseline_model()

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y_label, test_size=0.2, random_state=42)

In [42]:
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [43]:
model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE)

Epoch 1/200
11812/11812 [==============================] - 1s 122us/step - loss: 425.2554
Epoch 2/200
11812/11812 [==============================] - 1s 102us/step - loss: 277.3841
Epoch 3/200
11812/11812 [==============================] - 2s 179us/step - loss: 273.6978
Epoch 4/200
11812/11812 [==============================] - 1s 110us/step - loss: 271.4321
Epoch 5/200
11812/11812 [==============================] - 2s 130us/step - loss: 269.6850
Epoch 6/200
11812/11812 [==============================] - 2s 130us/step - loss: 268.2709
Epoch 7/200
11812/11812 [==============================] - 1s 114us/step - loss: 267.2019
Epoch 8/200
11812/11812 [==============================] - 1s 123us/step - loss: 266.2342
Epoch 9/200
11812/11812 [==============================] - 2s 135us/step - loss: 265.4295
Epoch 10/200
11812/11812 [==============================] - 2s 141us/step - loss: 264.7623
Epoch 11/200
11812/11812 [==============================] - 2s 138us/step - loss: 264.2275
Epoch 12

11812/11812 [==============================] - 1s 101us/step - loss: 260.3289
Epoch 91/200
11812/11812 [==============================] - 1s 103us/step - loss: 260.3419
Epoch 92/200
11812/11812 [==============================] - 1s 88us/step - loss: 260.3522
Epoch 93/200
11812/11812 [==============================] - 1s 89us/step - loss: 260.3270
Epoch 94/200
11812/11812 [==============================] - 1s 84us/step - loss: 260.3448
Epoch 95/200
11812/11812 [==============================] - 1s 86us/step - loss: 260.3683
Epoch 96/200
11812/11812 [==============================] - 1s 117us/step - loss: 260.0450
Epoch 97/200
11812/11812 [==============================] - 1s 111us/step - loss: 260.2824
Epoch 98/200
11812/11812 [==============================] - 2s 144us/step - loss: 260.3754
Epoch 99/200
11812/11812 [==============================] - 1s 97us/step - loss: 260.3293
Epoch 100/200
11812/11812 [==============================] - 1s 113us/step - loss: 260.2038
Epoch 101/200
11

11812/11812 [==============================] - 2s 151us/step - loss: 260.2808
Epoch 180/200
11812/11812 [==============================] - 1s 76us/step - loss: 260.3278
Epoch 181/200
11812/11812 [==============================] - 1s 83us/step - loss: 260.3021
Epoch 182/200
11812/11812 [==============================] - 1s 79us/step - loss: 260.3120
Epoch 183/200
11812/11812 [==============================] - 2s 209us/step - loss: 260.3526
Epoch 184/200
11812/11812 [==============================] - 1s 93us/step - loss: 260.2545
Epoch 185/200
11812/11812 [==============================] - 1s 87us/step - loss: 260.3102
Epoch 186/200
11812/11812 [==============================] - 1s 71us/step - loss: 260.2676
Epoch 187/200
11812/11812 [==============================] - 2s 138us/step - loss: 260.2601 0s - loss: 258
Epoch 188/200
11812/11812 [==============================] - 1s 81us/step - loss: 260.3313
Epoch 189/200
11812/11812 [==============================] - 1s 78us/step - loss: 260.

In [45]:
y_pred = model.predict(X_test)

y_pred = y_pred.flatten()

# how far off is the result? metric = mean squared error
y_test = np.array(y_test)
y_pred = np.array(y_pred)
print("Mean squared error: ", mean_squared_error(y_true=y_test, y_pred=y_pred))

# show first k differences
first_k = 10
print("Showing the first {} differences between predicted and real.".format(first_k))
for yp, yt in zip(y_pred[:first_k], y_test[:first_k]):
    print(yp, yt)

Mean squared error:  267.9923063549716
Showing the first 10 differences between predicted and real.
26.926544 56.0
23.011917 44.0
22.127241 6.0
20.630327 12.0
17.92175 45.0
19.586946 0.0
28.376467 17.0
21.01594 14.0
21.406012 10.0
18.977024 10.0


## Keras embedding + NN

### Generate input data 

In [46]:
X = []
for sample in w2v_corpus:
    input_sample = generate_input_matrix(sample, FIXED_LENGTH)
    if input_sample.shape != (FIXED_LENGTH,):
        print(input_sample.shape)
    X.append(input_sample)

### Encode the words

In [47]:
encoder = LabelEncoder()
encoder.fit(np.array(X).flatten())

LabelEncoder()

In [48]:
X_encoded = []
for sentence in X:
    encoded = encoder.transform(sentence)
    X_encoded.append(encoded)

### Split X in train and test

In [49]:
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y_label, test_size=0.2, random_state=42)

In [50]:
INPUT_SHAPE = (FIXED_LENGTH, )
INPUT_SHAPE

(17,)

### Define the Keras embedded model 

In [51]:
def keras_embedding_model(encoder):
    # Create model
    input_layer = Input(shape=(FIXED_LENGTH,), name='input')
    embedded_text = Embedding(input_dim=len(encoder.classes_),
                              output_dim=EMBEDDING_SIZE,
                              name='text_embedding')(input_layer)
    embedded_text = Flatten()(embedded_text)
    z = BatchNormalization()(embedded_text)
    z = Dense(32, activation='relu', name='fc1', kernel_initializer='normal')(z)
    z = BatchNormalization()(z)
    z = Dropout(0.7)(z)
    z = ReLU()(z)
    output = Dense(1, kernel_initializer='normal', activation='linear')(z)
    model = Model(inputs=[input_layer], outputs=[output])

    # Compile model
    model.compile(optimizer=Adam(lr=0.01), loss='mean_squared_error')

    return model

In [52]:
keras_embedded_model = keras_embedding_model(encoder=encoder)

In [53]:
keras_embedded_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 17)                0         
_________________________________________________________________
text_embedding (Embedding)   (None, 17, 50)            243150    
_________________________________________________________________
flatten_2 (Flatten)          (None, 850)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 850)               3400      
_________________________________________________________________
fc1 (Dense)                  (None, 32)                27232     
_________________________________________________________________
batch_normalization_2 (Batch (None, 32)                128       
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
__________

In [54]:
def schedule(epoch, lr):
    if epoch in [500]:
        return lr / 3.0
    return lr
scheduler = LearningRateScheduler(schedule=schedule)

In [55]:
y_series = pd.Series(y_label)
std_y, mean_y = y_series.describe()['std'], y_series.describe()['mean']

def rmse(y_true, y_pred):
    y_true = y_true * std_y + mean_y
    y_pred = y_pred * std_y + mean_y
    return K.sqrt(K.mean(K.square(y_true - y_pred), axis=0))

In [56]:
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [57]:
X_train.shape, y_train.shape

((11812, 17), (11812,))

In [58]:
history = keras_embedded_model.fit(X_train, y_train, epochs=EPOCHS,
                    callbacks=[scheduler, EarlyStopping(patience=10, monitor='rmse'),
                               ModelCheckpoint('models/embedding.h5', monitor='rmse')],
                    batch_size=BATCH_SIZE)

Epoch 1/200
11812/11812 [==============================] - 6s 512us/step - loss: 257.0772
Epoch 2/200
   16/11812 [..............................] - ETA: 5s - loss: 226.7228

/home/skadi/.local/lib/python3.5/site-packages/keras/callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `rmse` which is not available. Available metrics are: lr,loss
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


11812/11812 [==============================] - 5s 452us/step - loss: 179.5519
Epoch 3/200
11812/11812 [==============================] - 5s 446us/step - loss: 156.3318
Epoch 4/200
11812/11812 [==============================] - 5s 450us/step - loss: 143.7233
Epoch 5/200
11812/11812 [==============================] - 6s 466us/step - loss: 133.1381
Epoch 6/200
11812/11812 [==============================] - 6s 475us/step - loss: 124.7230
Epoch 7/200
11812/11812 [==============================] - 6s 473us/step - loss: 118.1138
Epoch 8/200
11812/11812 [==============================] - 6s 484us/step - loss: 114.9114
Epoch 9/200
11812/11812 [==============================] - 6s 486us/step - loss: 112.5876
Epoch 10/200
11812/11812 [==============================] - 6s 498us/step - loss: 108.0610
Epoch 11/200
11812/11812 [==============================] - 6s 475us/step - loss: 113.2753
Epoch 12/200
11812/11812 [==============================] - 6s 482us/step - loss: 106.1440
Epoch 13/200
11812/

Epoch 93/200
11812/11812 [==============================] - 5s 426us/step - loss: 83.0164
Epoch 94/200
11812/11812 [==============================] - 5s 443us/step - loss: 81.5588
Epoch 95/200
11812/11812 [==============================] - 5s 433us/step - loss: 81.0640
Epoch 96/200
11812/11812 [==============================] - 5s 427us/step - loss: 82.5015
Epoch 97/200
11812/11812 [==============================] - 5s 435us/step - loss: 80.4092
Epoch 98/200
11812/11812 [==============================] - 5s 445us/step - loss: 77.5473
Epoch 99/200
11812/11812 [==============================] - 5s 426us/step - loss: 82.4049
Epoch 100/200
11812/11812 [==============================] - 5s 420us/step - loss: 81.6600
Epoch 101/200
11812/11812 [==============================] - 5s 420us/step - loss: 77.2971
Epoch 102/200
11812/11812 [==============================] - 5s 418us/step - loss: 81.0036
Epoch 103/200
11812/11812 [==============================] - 5s 424us/step - loss: 81.1159
Epoch 

11812/11812 [==============================] - 5s 403us/step - loss: 77.1947
Epoch 184/200
11812/11812 [==============================] - 5s 405us/step - loss: 78.5959
Epoch 185/200
11812/11812 [==============================] - 5s 395us/step - loss: 78.3865
Epoch 186/200
11812/11812 [==============================] - 4s 331us/step - loss: 77.2839
Epoch 187/200
11812/11812 [==============================] - 4s 377us/step - loss: 77.0136
Epoch 188/200
11812/11812 [==============================] - 6s 489us/step - loss: 78.3555
Epoch 189/200
11812/11812 [==============================] - 5s 410us/step - loss: 78.9709
Epoch 190/200
11812/11812 [==============================] - 5s 429us/step - loss: 78.1522
Epoch 191/200
11812/11812 [==============================] - 4s 367us/step - loss: 77.3983
Epoch 192/200
11812/11812 [==============================] - 4s 351us/step - loss: 76.4492
Epoch 193/200
11812/11812 [==============================] - 4s 325us/step - loss: 76.8252
Epoch 194/200

In [61]:
y_pred = keras_embedded_model.predict(X_test)

y_pred = y_pred.flatten()

# how far off is the result? metric = mean squared error
y_test = np.array(y_test)
y_pred = np.array(y_pred)
print("Mean squared error: ", np.sqrt(mean_squared_error(y_true=y_test, y_pred=y_pred)))
print("Std: ", np.std(y_test))
print("Mean: ", np.mean(y_test))

# show first k differences
first_k = 10
print("Showing the first {} differences between predicted and real.".format(first_k))
for yp, yt in zip(y_pred[:first_k], y_test[:first_k]):
    print(yp, yt)

Mean squared error:  15.507113640836335
Std:  16.94692760867542
Mean:  19.83953960731212
Showing the first 10 differences between predicted and real.
57.28442 56.0
12.685392 44.0
15.537098 6.0
22.051819 12.0
29.747246 45.0
10.021058 0.0
19.196941 17.0
12.228628 14.0
13.779607 10.0
25.369438 10.0


### Save model

In [62]:
keras_embedded_model.save('models/predict_fat_model')

### Get the dataframe that has NaN values for our regression column (calories)

In [63]:
predict_df = df[df[regr_column] != df[regr_column]]

### Get the input data we need to predict the needed values

In [64]:
predict_data = []
for i in range(len(predict_df)):
    predict_data.append(get_small_dict(predict_df, keep_columns, i))
corpus = ['|'.join(
    d['tags'] + 
    [w.lower() for w in nltk.word_tokenize(d['title']) if w not in '''!()-[]{};:'"\,<>./?@#$%^&*_~''']
) for d in predict_data]

### Split input data using the tokenizer

In [65]:
predict_corpus = []
for word_list in corpus:
    predict_corpus.append(tokenize(word_list))

### Make sure input data has a fixed shape

In [66]:
X = []
for sample in predict_corpus:
    sample = [w for w in sample if w in encoder.classes_]
    input_sample = generate_input_matrix(sample, FIXED_LENGTH)
    if input_sample.shape != (FIXED_LENGTH,):
        print(input_sample.shape)
    X.append(input_sample)

### Transform matrix-like input data using available encoder

In [67]:
X_encoded = []
for sentence in X:
    encoded = encoder.transform(sentence)
    X_encoded.append(encoded)

In [68]:
X_encoded = np.array(X_encoded)
X_encoded.shape

(4183, 17)

### Predict missing data 

In [69]:
y_pred = keras_embedded_model.predict(X_encoded)

y_pred = y_pred.flatten()

### Fill the dataframe with predicted values

In [70]:
predict_df[regr_column] = y_pred

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


### Update initial dataframe with missing values

In [72]:
update_regr_column = [np.nan for _ in range(len(df))]
for idx, value in zip(predict_df.index, y_pred):
    update_regr_column[idx] = value

In [73]:
df.update({regr_column: update_regr_column})

In [74]:
df.head()

,title,rating,calories,protein,fat,sodium,#cakeweek,#wasteless,22-minute meals,3-ingredient recipes,...,yellow squash,yogurt,yonkers,yuca,zucchini,cookbooks,leftovers,snack,snack week,turkey
0,"Lentil, Apple, and Turkey Wrap",2.500,426.000000,30.000000,7.000000,559.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,Boudin Blanc Terrine with Red Onion Confit,4.375,403.000000,18.000000,23.000000,1439.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Potato and Fennel Soup Hodge,3.750,165.000000,6.000000,7.000000,165.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Mahi-Mahi in Tomato Olive Sauce,5.000,252.445801,14.630608,23.051666,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Spinach Noodle Casserole,3.125,547.000000,20.000000,32.000000,452.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Save updated dataframe

In [75]:
df.to_csv('recipes_with_calories_protein_and_fat.csv', sep='|')